# CUNY MSDS DATA618 - Quantative Finance

## Week 3 : Portfolio 

### Keeno Glanville
---------------------
You are a 37-year-old bond trader in a major investment firm. Your investment portfolio, largely holding high return equities is valued today at $6.2 M. You have just read Jamie Dimon's financial hurricane warning and decided to rebalance your portfolio.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import scipy.stats
from pandas_datareader import data
import yfinance as yf
from var import VaR
from pandas_datareader import data as pdr

In [ ]:
portfolio = pd.read_csv('https://raw.githubusercontent.com/kglan/MSDS/main/DATA618/portfolio.csv')
portfolio = portfolio[:-3]
portfolio.head(10)

In [ ]:

#Calculate weights 
# Convert the 'Price' column to a numeric value
tickers = ['AAPL', 'AMZN', 'MSFT', 'GOOGL', 'META', 'TSLA', 'NVDA', 'AMD', 'JPM', 'JNJ', 'V', 'PG', 'JCI', 'GE', 'KO', 'VZ', 'BA', 'T', 'PFE', 'NFLX', 'BABA', 'INTC', 'DIS', 'XOM', 'F', 'GS', 'HD', 'IBM', 'SBUX', 'CSCO', 'LQD', 'HYG', 'IEF', 'BND', 'SHY', 'VCSH', 'BKLN', 'TIP', 'SCHP', 'TLT']

portfolio['Price'] = portfolio['Price'].str.replace('$', '').str.replace(',', '').astype(float)

# Calculate the weight for each asset
portfolio['Amount'] = portfolio['Amount'].str.replace('$', '').str.replace(',', '').astype(float)
portfolio['weight'] = portfolio['Amount'] / portfolio['Amount'].sum()
portfolio.head()

In [ ]:
weights = portfolio['Amount'] / portfolio['Amount'].sum()
w = {ticker: weight for ticker, weight in zip(tickers, weights)}
first_5_values = {k: v for k, v in list(w.items())[:5]}
first_5_values

In [ ]:
yf.pdr_override()  # This sets Yahoo Finance as the data source for DataReader

# Date Range for portfolio
start_date = '2023-09-12'
end_date = '2023-10-08'


In [ ]:
data = pdr.DataReader(tickers, start=start_date, end=end_date)


In [ ]:
data['Close']


In [ ]:
pivoted = data['Close']
pivoted

# Portfolio Variance
A measuree of dispursion around an expected value

$$\sigma^2(R_p) = w_1w_1\sigma^2(R_1) + w_1w_2COV(R_1,R_2) + w_2w_2\sigma^2(R_2) + w_2w_1COV(R_1,R_2)$$
$$\sigma^2(R_p) = \sum_{i=1}^n\sum_{j=1}^nw_iw_jCOV(R_i, R_j)$$

In [ ]:
# Individual Asset Variance
returns =pivoted.pct_change()
var_ret=returns.var()
portfolio['Variance'] = var_ret.reindex(portfolio['Ticker']).values
var_ret

In [ ]:
# Individual Standard Deviation/Volatility

sd_ret = np.sqrt(var_ret)
portfolio['Volatility'] = sd_ret.reindex(portfolio['Ticker']).values
sd_ret

# Portfolio Variance with Correlation and Covariance
It is nice to try and get an understanding of the individual stock however we see that we want to get an understanding of the stock but we need to understand how our portfolio factors in

In [ ]:
cov_matrix= pivoted.pct_change().apply(lambda x: np.log(1+x)).cov()
cov_matrix

In [ ]:
# Portfolio Correlation Matrix
cov_matrix.mul(w, axis=0).mul(w,axis=1)

In [ ]:
p_var= cov_matrix.mul(w, axis=0).mul(w,axis=1).sum().sum()
p_var

# Portfolio Standard Deviation
$$\sigma(R_p) = \sqrt{\sigma^2(R_p)}$$

In [ ]:
# Portfolio Standard Deviation
p_sd= np.sqrt(p_var)
p_sd

In [ ]:
annual_p_sd= p_sd*np.sqrt(250)
annual_p_sd

# Portfolio Expected Return
In order to work with Expected Returns we must work with annual values

In [ ]:
a_start='2014-01-01'
a_end = '2023-10-08'
a_data = pdr.DataReader(tickers, start=a_start, end=a_end)
a_data = a_data['Close']

In [ ]:
df= a_data.resample('Y').last().pct_change()
df

In [ ]:
e_r = df.mean()
portfolio['Expected Return'] = e_r.reindex(portfolio['Ticker']).values

cash_index = portfolio[portfolio['Ticker'] == 'Cash'].index[0]
portfolio.at[cash_index, 'Expected Return'] = 0
portfolio.at[cash_index, 'Variance'] = 0
portfolio.at[cash_index, 'Volatility'] = 0

e_r


In [ ]:
# Portfolio Expected Return
p_e_r=(e_r* pd.Series(w)).sum()
p_e_r

In [ ]:
expected_return = portfolio['Expected Return']
volatility = portfolio['Volatility']
tickerz = portfolio['Ticker']
# Create a scatter plot
plt.figure(figsize=(15, 6))
plt.scatter(volatility, expected_return, c=portfolio['Subtype'].factorize()[0], cmap='viridis', marker='o', s=100)


# Label the points with the asset ticker or name
for i, ticker in enumerate(tickerz):
    plt.annotate(ticker, (volatility[i], expected_return[i]), textcoords="offset points", xytext=(0, 10), ha='center')

plt.title('Expected Return vs. Volatility')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
expected_return = portfolio['Expected Return']
volatility = portfolio['Volatility']
tickerz = portfolio['Ticker']
# Create a scatter plot
plt.figure(figsize=(15, 6))
plt.scatter(volatility, expected_return, c=portfolio['Subtype'].factorize()[0], cmap='viridis', marker='o', s=100)


# Label the points with the asset ticker or name
for i, ticker in enumerate(tickerz):
    plt.annotate(ticker, (volatility[i], expected_return[i]), textcoords="offset points", xytext=(0, 10), ha='center')

plt.title('Expected Return vs. Volatility')
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.grid(True)


plt.scatter(p_sd, p_e_r, color='red', marker='o', s=100)
plt.annotate("Portfolio", (p_sd, p_e_r), textcoords="offset points", xytext=(0, 10), ha='center', color='red')

# Show the plot
plt.show()

# Sharpe Ratio
$$Sharpe Ratio = \frac{E(R_i)-rf}{\sigma_i}$$

In [ ]:
# Risk free rate utilizing US treasuery yield -> 5.47%
rf=0.0547

In [ ]:
portfolio['Sharpe'] = portfolio['Expected Return'].sub(rf).div(portfolio['Volatility'])
portfolio.head()

In [ ]:

# Create a scatter plot of Sharpe Ratios
plt.figure(figsize=(15, 6))
plt.scatter(portfolio['Volatility'], portfolio['Expected Return'], c=portfolio['Sharpe'], cmap='viridis', s=100)
plt.xlabel('Volatility')
plt.ylabel('Expected Return')
plt.title('Expected Return vs. Volatility with Sharpe Ratios')
plt.colorbar(label='Sharpe Ratio')
plt.grid(True)

# Label points with the asset ticker
for i, ticker in enumerate(portfolio['Ticker']):
    plt.annotate(ticker, (portfolio['Volatility'][i], portfolio['Expected Return'][i]), textcoords="offset points", xytext=(0, 10), ha='center')

plt.show()

In [ ]:
# Create a scatter plot of Sharpe Ratios vs. Asset Weights
plt.figure(figsize=(15, 6))
plt.scatter(portfolio['weight'], portfolio['Sharpe'], c=portfolio['Sharpe'],cmap='viridis', s=100)
plt.xlabel('Asset Weight')
plt.ylabel('Sharpe Ratio')
plt.title('Sharpe Ratio vs. Asset Weight')
plt.grid(True)

# Label points with the asset ticker
for i, ticker in enumerate(portfolio['Ticker']):
    plt.annotate(ticker, (portfolio['weight'][i], portfolio['Sharpe'][i]), textcoords="offset points", xytext=(0, 10), ha='center')

plt.show()

# Volume Weighted Average Price (VWAP)

In [ ]:
vwap_high =  data['High']
vwap_high

In [ ]:
vwap_low = data['Low']
vwap_low

In [ ]:
vwap_open= data['Open']

In [ ]:
vwap_close = data['Close']
vwap_close

In [ ]:
vwap_vol = data['Volume']
vwap_vol

In [ ]:
vwap_typ = (vwap_high + vwap_low + vwap_close)/3
vwap_typ

In [ ]:
vwap_money_flow = vwap_typ * vwap_vol
vwap_money_flow

In [ ]:
vwap_cumulative_money_flow = vwap_money_flow.cumsum()
vwap_cumulative_volume = vwap_vol.cumsum()


In [ ]:
vwap = vwap_cumulative_money_flow / vwap_cumulative_volume
vwap

In [ ]:
# Select the stocks you want to plot (TSLA, AAPL, BND, and TIP)
selected_stocks = ['TSLA', 'AAPL', 'BND', 'TIP', 'SHY']

# Create individual line charts for each selected stock
for stock in selected_stocks:
    plt.figure(figsize=(10, 6))
    plt.plot(vwap.index, vwap[stock])
    plt.xlabel('Date')
    plt.ylabel('VWAP')
    plt.title(f'VWAP for {stock}')
    plt.grid()
    plt.show()


In [ ]:
for stock in selected_stocks:
    plt.figure(figsize=(10, 6))
    
    # Create a DataFrame for the selected stock's data
    stock_data = pd.DataFrame({
        'Open': vwap_open[stock],
        'High': vwap_high[stock],
        'Low': vwap_low[stock],
        'Close': vwap_close[stock],
        'VWAP': vwap[stock]
    }, index=vwap.index)
    
    # Plot the data using Seaborn
    sns.lineplot(data=stock_data, dashes=False)
    
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(f'Price and VWAP Chart for {stock}')
    plt.legend()
    plt.show()

In [ ]:
import mplfinance as mpf

# Create individual candlestick charts for each selected stock
for stock in selected_stocks:
    ohlc_data = pd.DataFrame({
        'Open': vwap_open[stock],
        'High': vwap_high[stock],
        'Low': vwap_low[stock],
        'Close': vwap_close[stock]
    }, index=vwap.index)
    
    mpf.plot(ohlc_data, type='candle', title=f'Candlestick Chart for {stock}', ylabel='Price', figsize=(10, 6))
    plt.show()

# Alpha

### Spy benchmark

In [ ]:
start_date = '2023-09-12'
end_date = '2023-10-08'
bench = pdr.DataReader('^GSPC', start=start_date, end=end_date)
bench

In [ ]:
# Change
b_start='2014-01-01'
b_end = '2023-10-08'
b_data = pdr.DataReader('^GSPC', start=b_start, end=b_end)
b_data = b_data['Close']
b_df= b_data.resample('Y').last().pct_change()
b_df
b_r = b_df.mean()
b_r
#portfolio['Expected Return'] = e_r.reindex(portfolio['Ticker']).values

In [ ]:
#benchmark returns annually
b_df

In [ ]:
#portfolio returns annual
df_pr = df.mean(axis=1)
df_pr

In [ ]:
import statsmodels.api as sm

# Remove rows with NaN values
df = df.dropna()
b_df = b_df.dropna()

# Define the independent variable (benchmark returns)
X = b_df

# Add a constant for the intercept
X = sm.add_constant(X)

# Define the dependent variable (portfolio returns)
Y = df_pr
Y = Y.reindex(X.index)
# Fit the regression model
model = sm.OLS(Y, X).fit()

# Get the regression summary
print(model.summary())

In [ ]:
#Alpha = (Portfolio Return - Benchmark Return) - Risk-Free Rate
beta = model.params['Close']
beta

In [ ]:
alpha = p_e_r - (rf + beta * (b_r - rf))
alpha